#### -----------------------------------------------------------------------------<br>Copyright (c) 2023, Lucid Vision Labs, Inc.
##### THE  SOFTWARE  IS  PROVIDED  "AS IS",  WITHOUT  WARRANTY  OF  ANY  KIND,<br>EXPRESS  OR  IMPLIED,  INCLUDING  BUT  NOT  LIMITED  TO  THE  WARRANTIES<br>OF  MERCHANTABILITY,  FITNESS  FOR  A  PARTICULAR  PURPOSE  AND<br>NONINFRINGEMENT.  IN  NO  EVENT  SHALL  THE  AUTHORS  OR  COPYRIGHT  HOLDERS<br>BE  LIABLE  FOR  ANY  CLAIM,  DAMAGES  OR  OTHER  LIABILITY,  WHETHER  IN  AN<br>ACTION  OF  CONTRACT,  TORT  OR  OTHERWISE,  ARISING  FROM,  OUT  OF  OR  IN<br>CONNECTION  WITH  THE  SOFTWARE  OR  THE  USE  OR  OTHER  DEALINGS  IN  THE  SOFTWARE.<br>-----------------------------------------------------------------------------

In [1]:
import ctypes  # ctypes.cast(), ctypes.POINTER(), ctypes.c_ushort
import os  # os.getcwd()
import time
from pathlib import Path

import numpy as np  # pip install numpy
from PIL import Image as PIL_Image  # pip install Pillow

import time
from arena_api.system import system
from datetime import datetime

##### Exposure: Introduction
>    This example introduces the exposure feature. An image's exposure
    time refers to the amount of time that a device's sensor is exposed
    to a scene before the data is collected. The exposure can be
    handled automatically or manually.

In [2]:
TAB1 = "  "
TAB2 = "    "
num_images = 1
aq_frame_rate = 0.1 # Hz
exposure_time = 1.0 * 10 ** 5 # us
gain = 25.0 # dB
timeout = 2000

In [3]:
"""
Waits for the user to connect a device before raising an exception if it fails
"""
tries = 0
tries_max = 6
sleep_time_secs = 10
devices = None
while tries < tries_max:  # Wait for device for 60 seconds
    devices = system.create_device()
    if not devices:
        print(
            f'Try {tries+1} of {tries_max}: waiting for {sleep_time_secs} '
            f'secs for a device to be connected!')
        for sec_count in range(sleep_time_secs):
            time.sleep(1)
            print(f'{sec_count + 1 } seconds passed ',
                    '.' * sec_count, end='\r')
        tries += 1
    else:
        break
else:
    raise Exception(f'No device found! Please connect a device and run '
                    f'the example again.')

device = devices[0]
nodemap = device.nodemap
tl_stream_nodemap = device.tl_stream_nodemap
print(f'Device used in the example:\n{TAB1}{device}')

Device used in the example:
  ('1c:0f:af:13:0d:71', 'ATX081S-U', '', '169.254.114.13')


##### Obtain relevant camera parameters
> Parameters we may want to vary for a given measurement. Disable automatic exposure time, automatic gain, and automatic frame rate.

In [4]:
nodes = nodemap.get_node(['AcquisitionFrameRateEnable', 'AcquisitionFrameRate', 
                          'ExposureAuto', 'ExposureTime', 'DeviceIndicatorMode',
                          'PixelFormat', 'GainAuto', 'Gain'])

nodes['ExposureAuto'].value = 'Off'
nodes['GainAuto'].value = 'Off'
nodes['AcquisitionFrameRateEnable'].value = True
nodes['PixelFormat'].value = 'Mono16'
nodes['DeviceIndicatorMode'].value = 'Inactive'

##### Set frame rate, exposure time, and gain.
>   Before setting these variables, check that they are not outside
    of the acceptable range. If any are, return an error.

In [5]:
def set_node_val(node_string, new_val):
    min_val = nodes[node_string].min
    max_val = nodes[node_string].max
    if (new_val > max_val or new_val < min_val):
        raise ValueError(node_string + ' must be between ' +
                         format(min_val, '3.2f') + ' and ' +
                         format(max_val, '3.2f'))
    else:
        nodes[node_string].value = new_val

set_node_val('AcquisitionFrameRate', aq_frame_rate)
set_node_val('ExposureTime', exposure_time)
set_node_val('Gain', gain)

##### Setup stream values

In [6]:
tl_stream_nodemap = device.tl_stream_nodemap
tl_stream_nodemap['StreamAutoNegotiatePacketSize'].value = True
tl_stream_nodemap['StreamPacketResendEnable'].value = True
# object_methods = [name for name in dir(device) if callable(getattr(device, name))]

##### Start stream and grab images

In [7]:
exposure_time_list = [3.0 * 10 ** 6]
num_images = len(exposure_time_list)
filter = '9b'

print(f"{TAB1}Getting {num_images} images")

for i, exposure_time in enumerate(exposure_time_list):
    set_node_val('ExposureTime', exposure_time)
    device.start_stream(1)
    buffer = device.get_buffer()

    print(f'{TAB2}Buffer {i} received | '
            f'Timestamp ({buffer.timestamp_ns} ns)')
    end_time = str(datetime.now().hour) + '_' + str(datetime.now().minute) + \
               '_' + str(datetime.now().second)
    print('Converting image buffer to a numpy array')

    pdata_as16 = ctypes.cast(buffer.pdata,
                            ctypes.POINTER(ctypes.c_ushort))
    nparray_reshaped = np.ctypeslib.as_array(
        pdata_as16,
        (buffer.height, buffer.width))

    # Saving --------------------------------------------------------------
    print('Saving image')

    png_name = 'filter' + filter + '_' + end_time + '.png'
    nparray_reshaped_as_bytes = nparray_reshaped.tobytes()
    png_array = PIL_Image.new('I', nparray_reshaped.T.shape)
    png_array.frombytes(nparray_reshaped_as_bytes, 'raw', 'I;16')

    png_array.save(os.getcwd() + '\\..\\Saved Images\\QE\\Dark Images\\' + png_name)

    device.requeue_buffer(buffer)
    device.stop_stream()

  Getting 1 images
    Buffer 0 received | Timestamp (6250255516184 ns)
Converting image buffer to a numpy array
Saving image


##### Clean up ----------------------------------------------------------------
> - Destroy device. This call is optional and will automatically be
  called for any remaining devices when the system module is unloading.


In [8]:
system.destroy_device(device)